In [6]:
from typing import Dict, Text

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [5]:
!pip install -q --upgrade tensorflow-datasets

In [1]:
!pip install tensorflow-recommenders

     |████████████████████████████████| 127 kB 6.9 MB/s eta 0:00:01


In [3]:
!pip install tensorflow

     |████████████████████████████████| 320.4 MB 12 kB/s s eta 0:00:01        | 133.8 MB 73.8 MB/s eta 0:00:03        | 137.6 MB 73.8 MB/s eta 0:00:03�█████████▊                 | 147.1 MB 73.8 MB/s eta 0:00:03��▊               | 167.4 MB 73.8 MB/s eta 0:00:03�               | 170.5 MB 73.8 MB/s eta 0:00:03███▎        | 232.9 MB 39.6 MB/s eta 0:00:03��████████████████▎       | 243.0 MB 41.2 MB/s eta 0:00:02�█▊ | 307.2 MB 50.5 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.6 MB/s  eta 0:00:01
     |████████████████████████████████| 6.8 MB 55.4 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 82.9 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 779 kB 47.1 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=e59a69855bda5878e04f7621c5a81d47d1cb3658c359570de733

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sp
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse.linalg import spsolve

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from pathlib import Path
import pandas as pd
import numpy as np
import random
from IPython.display import Image
from IPython.core.display import HTML 
import pdb;

from subprocess import check_output
from sklearn.model_selection import train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

# Importing Libraries and cookbooks
from recsys import *## recommender system cookbook
#from generic_preprocessing import * ## pre-processing code
from IPython.display import HTML ## Setting display options for Ipython Notebook
from itertools import combinations, chain, product

In [8]:
demo = pd.read_csv("demo001.csv",sep="|")

demo.head(10)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID_CTE,FECHA_NAC,GENERO,EDO_CIVIL,D_EDO,DESC_D_EDO,D_NC,DESC_D_NC,TP_HOGAR,T_HOGAR,...,FECHADIFCOBRO,SDOCTAPERDIDA,FECHACORTE,TIPOTELEFONO,NUMEROTELEFONOORIGEN,NUMEROTELEFONO,CARRIER,DES_CORREOELECTRONICO,DES_ORIGEN,SAMPLING
0,2746824,1952-09-22 00:00:00.000000,F,C,9.0,AGUASCALIENTES,36.0,AGUASCALIENTES,P,1988-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,1.0,9.130889e+06,4.499131e+09,0.0,CONCHITAGUILAR2@GMAIL.COM,Coppel.com en tienda,0.004464
1,6572723,1958-11-04 00:00:00.000000,F,C,10.0,JALISCO,67.0,TONALA,P,1992-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,4.0,6.170833e+06,3.336171e+09,0.0,MOGLEZZB1102@HOTMAIL.COM,Tienda,0.005916
2,12334731,1980-03-14 00:00:00.000000,M,C,23.0,VERACRUZ,40.0,CORDOBA,R,2012-01-01 00:00:00.000000,...,2011-08-31 00:00:00.000000,0.0,2020-07-31 00:00:00.000000,1.0,2.711111e+09,2.711111e+09,0.0,JORGECONTRERAS_08@HOTMAIL.COM,Tienda,0.003026
3,12408172,1968-02-19 00:00:00.000000,M,S,18.0,PUEBLA,85.0,ATLIXCO,P,2007-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,1.0,0.000000e+00,2.444463e+09,0.0,MARIO_1018@HOTMAIL.COM,Coppel.com,0.002311
4,24428035,1950-03-18 00:00:00.000000,M,D,25.0,DISTRITO FEDERAL,107.0,IZTAPALAPA,F,2003-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,3.0,5.541973e+09,5.541973e+09,0.0,BARBIE_COMEX@HOTMAIL.COM,CAT,0.008750
5,42979358,1963-10-12 00:00:00.000000,M,S,15.0,ZACATECAS,273.0,SOMBRERETE,P,1978-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,2.0,4.331055e+09,4.331055e+09,39.0,B.LINDA.MORALES@GMAIL.COM,Tienda,0.007626
6,49727061,1970-02-17 00:00:00.000000,F,C,4.0,BAJA CALIFORNIA NORTE,64.0,ENSENADA,P,2010-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,2.0,5.562271e+09,5.562271e+09,39.0,LOLISMAGA_24@HOTMAIL.COM,Tienda,0.006580
7,26835752,1984-06-14 00:00:00.000000,F,C,23.0,VERACRUZ,183.0,TUXPAN,P,2013-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,2.0,7.681044e+09,7.681044e+09,39.0,NaN,NaN,0.007558
8,25066445,1974-10-12 00:00:00.000000,F,C,22.0,HIDALGO,316.0,MINERAL DE LA R,P,2019-12-09 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,2.0,5.520714e+09,5.520714e+09,39.0,NaN,NaN,0.009830
9,46495652,1986-11-02 00:00:00.000000,M,S,31.0,OAXACA,384.0,LOMA BONITA,P,2004-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,5.0,2.871169e+09,2.871169e+09,0.0,NaN,NaN,0.002984


In [9]:
df_demo=demo.loc[:,('ID_CTE', "GENERO", 'EDO_CIVIL')]
df_demo.head(10)

,ID_CTE,GENERO,EDO_CIVIL
0,2746824,F,C
1,6572723,F,C
2,12334731,M,C
3,12408172,M,S
4,24428035,M,D
5,42979358,M,S
6,49727061,F,C
7,26835752,F,C
8,25066445,F,C
9,46495652,M,S


In [10]:
tran = pd.read_csv("transactional001.csv",sep="|")

tran.head(10)

,ID_CTE,FECHA_TICKET,ID_TIENDA,ID_TICKET,PROD_SKU,PROD_DEP,DESC_PROD_DEP,PROD_CLAS,DESC_PROD_CLAS,PROD_FAM,DESC_PROD_FAM,PROD_AREA,PROD_CATEGORIA,DESC_PROD_CATEGORIA,IMPORTE_VTA,CANTIDAD_VTA,PROD_PRECIO_PROMEDIO,TIPOCOMPRA,FECHACORTE
0,145035,2017-10-02 00:00:00.000000,3,133548,-99,-99,Sin Descripcion,-99,Sin Descripcion,-99,Sin Descripcion,Prestamos,-99,Sin Descripcion,3448.00,1,3448.00,Credito,2017-10-31 00:00:00.000000
1,145035,2018-05-08 00:00:00.000000,1252,7400,-99,-99,Sin Descripcion,-99,Sin Descripcion,-99,Sin Descripcion,Prestamos,-99,Sin Descripcion,5646.00,1,5646.00,Credito,2018-05-31 00:00:00.000000
2,145035,2019-08-23 00:00:00.000000,3,48708,215063,2,ELECTRONICA,24,TELEFONOS CELULARES,85,AT.T,Muebles,1,Celulares,2154.32,1,2154.32,Credito,2019-08-31 00:00:00.000000
3,145035,2019-09-20 00:00:00.000000,5,23998,800404,8,ZAPATERIA,60,ZAPATO DAMA T 22.5-27,48,SENORA CONFORT,Ropa,17,Zapato Dama,603.00,1,603.00,Credito,2019-09-30 00:00:00.000000
4,145035,2020-01-13 00:00:00.000000,2,171365,-99,-99,Sin Descripcion,-99,Sin Descripcion,-99,Sin Descripcion,Prestamos,-99,Sin Descripcion,17241.00,1,17241.00,Credito,2020-01-31 00:00:00.000000
5,558561,2017-03-17 00:00:00.000000,119,505516,340634,3,MUEBLE SUELTO,30,JARDIN,11,MESA PLEGABLE,Muebles,3,Hogar,1119.83,1,1119.83,Credito,2017-03-31 00:00:00.000000
6,558561,2017-10-17 00:00:00.000000,119,526127,236785,2,ELECTRONICA,29,CHIPS CELULARES,11,CHIP 4G MULTIFORMATO,Muebles,1,Celulares,0.00,1,0.00,Contado,2017-10-31 00:00:00.000000
7,558561,2017-10-17 00:00:00.000000,119,526127,236905,2,ELECTRONICA,95,ACCESORIOS CELULARES,2,MICAS,Muebles,1,Celulares,128.45,1,128.45,Credito,2017-10-31 00:00:00.000000
8,558561,2017-10-17 00:00:00.000000,119,526127,253311,2,ELECTRONICA,24,TELEFONOS CELULARES,9,TELCEL R2,Muebles,1,Celulares,2671.55,1,2671.55,Credito,2017-10-31 00:00:00.000000
9,558561,2017-10-17 00:00:00.000000,119,526127,296475,2,ELECTRONICA,95,ACCESORIOS CELULARES,27,PROTECTOR DE PANTALLA VIDRIO,Muebles,1,Celulares,171.55,1,171.55,Credito,2017-10-31 00:00:00.000000


In [11]:
df_tran=tran[(tran["PROD_AREA"]=="Muebles")  | (tran["PROD_AREA"]=="Ropa")]


In [238]:
df_tran["area"]=df_tran["PROD_AREA"].apply(lambda x: 1 if x == 'Ropa' else 2) 
df_tran["fam"]=df_tran["area"].astype(str)+df_tran["PROD_DEP"].astype(str)+df_tran["PROD_CLAS"].astype(str).str.zfill(2)+df_tran["PROD_FAM"].astype(str).str.zfill(3)
df_tran["movie_id"]=df_tran["fam"].astype(str)
df_tran["user_id"]=df_tran["ID_CTE"].astype(str)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [186]:
df_tran.dtypes

ID_CTE                    int64
FECHA_TICKET             object
ID_TIENDA                 int64
ID_TICKET                 int64
PROD_SKU                  int64
PROD_DEP                  int64
DESC_PROD_DEP            object
PROD_CLAS                 int64
DESC_PROD_CLAS           object
PROD_FAM                  int64
DESC_PROD_FAM            object
PROD_AREA                object
PROD_CATEGORIA            int64
DESC_PROD_CATEGORIA      object
IMPORTE_VTA             float64
CANTIDAD_VTA              int64
PROD_PRECIO_PROMEDIO    float64
TIPOCOMPRA               object
FECHACORTE               object
area                      int64
fam                      object
ID_FAM                   object
user_id                   int64
movie_id                  int64
dtype: object

In [239]:
df_f=df_tran.loc[:,('user_id', 'FECHA_TICKET', 'movie_id')].groupby(['user_id',"movie_id"]).size().reset_index()
df_f.head(10)

,user_id,movie_id,0
0,10000040,1105068,1
1,10000040,1204616,1
2,10000040,1701216,2
3,10000040,1863254,2
4,10000212,1106055,3
5,10000212,1704352,1
6,10000212,1867308,2
7,10000212,1869214,1
8,10000212,2224057,1
9,10000212,2229011,1


In [188]:
df_f.dtypes

user_id     int64
movie_id    int64
0           int64
dtype: object

In [240]:
dff=df_f["movie_id"]
dff.head(5)

0    1105068
1    1204616
2    1701216
3    1863254
4    1106055
Name: movie_id, dtype: object

In [241]:
dfff=df_f.loc[:,("user_id","movie_id")]
dfff.head(5)

,user_id,movie_id
0,10000040,1105068
1,10000040,1204616
2,10000040,1701216
3,10000040,1863254
4,10000212,1106055


In [192]:
target = dfff.pop("movie_id")

In [242]:
ratings = tf.data.Dataset.from_tensor_slices((dfff.to_dict('list')))

In [243]:
#ratings = tf.data.Dataset.from_tensor_slices((dfff.values))

In [244]:
ratings.take(10)

<TakeDataset shapes: {user_id: (), movie_id: ()}, types: {user_id: tf.string, movie_id: tf.string}>

In [245]:
ratings

<TensorSliceDataset shapes: {user_id: (), movie_id: ()}, types: {user_id: tf.string, movie_id: tf.string}>

In [246]:
dfr = tfds.as_dataframe(ratings.take(10))
dfr.head()

,movie_id,user_id
0,b'1105068',b'10000040'
1,b'1204616',b'10000040'
2,b'1701216',b'10000040'
3,b'1863254',b'10000040'
4,b'1106055',b'10000212'


In [247]:
dfr.columns

Index(['movie_id', 'user_id'], dtype='object')

In [248]:
movies= tf.data.Dataset.from_tensor_slices((dff.values))

In [249]:
movies

<TensorSliceDataset shapes: (), types: tf.string>

In [250]:
dfm = tfds.as_dataframe(movies.take(10))
dfm.head()

,
0,b'1105068'
1,b'1204616'
2,b'1701216'
3,b'1863254'
4,b'1106055'


In [155]:
# Ratings data.
#ratings = tfds.load("ratings", split="train")
# Features of all the available movies.
#movies = tfds.load('movie_lens/100k-movies', split="train")

In [178]:
# Select the basic features.ID_CTE	ID_FAM
ratings1 = ratings.map(lambda x, y: {"user_id": col1 ,
    "movie_id": col2 })
#movies = movies.map(lambda x: tf.strings.to_number(x["movie_id"]))

TypeError: in user code:


    TypeError: <lambda>() missing 1 required positional argument: 'y'


In [ ]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
#y=ratings.map(lambda x: x["user_id"])

#movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
#movie_titles_vocabulary.adapt(movies)

In [235]:
dfm = tfds.as_dataframe(y.take(10))
dfm.head()

,
0,10366
1,10366
2,10366
3,10366
4,10366


In [231]:
# Build a model.
class Model(tfrs.Model):

  def __init__(self):
    super().__init__()

    # Set up user representation.
    self.user_model = tf.keras.layers.Embedding(
        input_dim=2000, output_dim=64)
    # Set up movie representation.
    self.item_model = tf.keras.layers.Embedding(
        input_dim=2000, output_dim=64)
    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.item_model(features["movie_id"])

    return self.task(user_embeddings, movie_embeddings)


model = Model()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [232]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [233]:
# Train.
model.fit(train.batch(4096), epochs=5)

Epoch 1/5


InvalidArgumentError:  indices[3840] = 1189246 is not in [0, 2000)
	 [[node embedding_4/embedding_lookup (defined at <ipython-input-231-d5fdf6db76dc>:23) ]] [Op:__inference_train_function_2043]

Errors may have originated from an input operation.
Input Source operations connected to node embedding_4/embedding_lookup:
 IteratorGetNext (defined at <ipython-input-233-bb0fc96a9c10>:2)

Function call stack:
train_function


In [73]:
# Evaluate.
model.evaluate(test.batch(4096), return_dict=True)

TypeError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1208 run_step  **
        outputs = model.